# Topic


<a id="cont"></a>

## Table of Contents
* <b>[1. Project Overview](#chapter1)
    * [1.1 Introduction](#section_1_1)
       * [1.1.1 Problem Statement](#sub_section_1_1_1)
* <b>[2. Data Overview](#chapter2)   
* <b>[3. Importing Packages](#chapter3)
* <b>[4. Loading Data](#chapter4)
* <b>[5. Data Cleaning](#chapter5)
* <b>[6. Exploratory Data Analysis (EDA)](#chapter6)


#[1. Project Overview](#chapter1)
## [1.1 Introduction](#section_1_1)

In today's digital era, recommender systems play a crucial role in helping people find content that suits their interests. Platforms like Netflix, Amazon Prime, Showmax, and Disney rely on sophisticated algorithms to suggest movies and shows tailored to individual preferences. But have you ever wondered how these platforms seem to know your tastes so well?

This project aims to develop a recommender system specifically for anime titles, leveraging advanced machine learning techniques. The system will help anime fans discover new shows that match their unique preferences, making the viewing experience more enjoyable and personalized.

### [1.1.1 Problem Statement](#sub_section_1_1_1)

The primary goal of this project is to create a recommender system that can accurately predict how users will rate anime titles they haven't watched yet. We will achieve this by combining collaborative filtering and content-based filtering methods, using a comprehensive dataset from myanimelist.net.

Key challenges we will address include:

1. **Data Analysis and Preparation:** Cleaning and organizing the dataset to ensure it's suitable for building our models.
2. **Collaborative Filtering:** Developing a model that recommends anime based on the preferences of users with similar tastes.
3. **Content-Based Filtering:** Creating a model that suggests anime based on the characteristics of the titles a user has already enjoyed.
4. **Hybrid Approach:** Integrating both collaborative and content-based filtering methods to enhance recommendation accuracy.
5. **Evaluation and Optimization:** Testing our system and fine-tuning it to provide the best possible recommendations.

By tackling these challenges, we'll create a powerful recommender system that not only helps users discover new anime but also enhances their overall viewing
experience. This project highlights the impact of machine learning in revolutionizing how we find and enjoy content in the entertainment industry.

# [2. Data Overview](#chapter2)

This project utilizes a comprehensive dataset to develop our anime recommender system. The dataset includes information about various anime content, such as movies, television series, music, specials, OVAs (Original Video Animations), and ONAs (Original Net Animations). The data is divided into several files:

1. **anime.csv**: Contains details about each anime.

2. **train.csv**: Contains user rating data for individual anime titles.

3. **test.csv**: Used for creating the final submission.


The dataset can be accessed [here](https://www.kaggle.com/competitions/anime-recommender-system-project-2024/data)

By analyzing and utilizing this dataset, we aim to create a robust recommender system that helps users discover new anime titles that align with their preferences.


# [3. Importing Packages](#chapter3)


In [73]:
import pandas as pd
import numpy as np
import re
import html
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


# [4. Loading Data](#chapter4)

In [ ]:
# Load and Define the datasets
anime_df = pd.read_csv('anime.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Exploring the Dataset

In this section, we will explore the dataset to understand its structure and the information it contains. The code snippets below provide an overview of the dataset's shape, the number of unique categories in the 'category' column, and metadata information about the dataset.

In [ ]:
# Verify the loaded datasets
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
train_df.head()

,user_id,anime_id,rating
0,1,11617,10
1,1,11757,10
2,1,15451,10
3,2,11771,10
4,3,20,8


In [ ]:
# Inspect the anime dataset
print(anime_df.info())
print(anime_df.describe())

# Inspect the train dataset
print(train_df.info())
print(train_df.describe())

# Inspect the test dataset
print(test_df.info())
print(test_df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None
           anime_id        rating       members
count  12294.000000  12064.000000  1.229400e+04
mean   14058.221653      6.473902  1.807134e+04
std    11455.294701      1.026746  5.482068e+04
min        1.000000      1.670000  5.000000e+00
25%     3484.250000      5.880000  2.250000e+02
50%    10260.500000      6.570000  1.550000e+03
75%    24794.500000      7.180000  9.437000e+03
max    34527.000000     10.000000  1.013917e+06
<class 'pandas.core.frame.DataFrame'>
Ra

# [5. Data Cleaning](#chapter5)

Data data cleaning, refers to the process of identifying and correcting errors, inconsistencies, and inaccuracies in a dataset to improve its quality and reliability for analysis or other purposes. It involves several tasks aimed at ensuring that the data is accurate, complete, and consistent.

It is an essential step in the data analysis process as it helps improve the quality, reliability, and usability of the data, leading to more accurate and insightful analysis results. Below are some common tasks involved in data clean up:

1. Check for Missing Values AND Handle Missing Values: identify missing values in the DataFrame and you can choose to drop rows with missing values, fill them with a specific value, or use more advanced imputation techniques.
2. Check for Duplicates AND Handle Duplicates: Identify duplicate rows in the DataFrame and if they are found, you can choose to drop them.
3. Decode HTML entities
4. Ensure correct data types
5. Remove non-alphanumeric characters
6. Convert text columns to lowercases




In [ ]:
#1 Check for missing values
print(anime_df.isnull().sum())
print(train_df.isnull().sum())
print(test_df.isnull().sum())

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64
user_id     0
anime_id    0
rating      0
dtype: int64
user_id     0
anime_id    0
dtype: int64


The information above comfirms that there are missing values in the anime.csv dataset and none in the train.csv and test.csv, therefor we need to handle the missing values in the anime.csv appropriately to ensure the dataset is clean and ready for analysis. Here are the steps to handle the missing values for each column:

* genre: Impute missing values with the most common genre or a placeholder value.

* type: Impute missing values with the most common type or a placeholder value.

* rating: Impute missing values with the average rating.


In [ ]:
# Handle missing values in 'genre' by imputing with the most common genre
most_common_genre = anime_df['genre'].mode()[0]
anime_df['genre'].fillna(most_common_genre, inplace=True)

# Handle missing values in 'type' by imputing with the most common type
most_common_type = anime_df['type'].mode()[0]
anime_df['type'].fillna(most_common_type, inplace=True)

# Handle missing values in 'rating' by imputing with the average rating
mean_rating = anime_df['rating'].mean()
anime_df['rating'].fillna(mean_rating, inplace=True)

# Verify that there are no more missing values
print(anime_df.isnull().sum())



anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


In [ ]:
#2 Check for duplicates based on all columns
print(anime_df.duplicated().sum())
print(train_df.duplicated().sum())
print(test_df.duplicated().sum())

0
1
0


The information above comfirms that there is 1 duplicate in the train_df and none in the anime_df and test_df, therefor we need to handle the duplicate in the train_df appropriately to ensure the dataset is clean and ready for analysis. Here is the step to handle the duplicate:

* Remove the duplicate by dropping it

In [ ]:
# Remove duplicates
train_df.drop_duplicates(inplace=True)
# Verify that there are no more duplicates
print(anime_df.duplicated().sum())


0


In [82]:
#3 Clean the 'name' column by decoding HTML entities
anime_df['name'] = anime_df['name'].apply(html.unescape)

#4 Ensure correct data types
anime_df['rating'] = anime_df['rating'].astype(float)
anime_df['members'] = anime_df['members'].astype(int)

#5 Remove non-alphanumeric characters from the 'name' column
anime_df['name'] = anime_df['name'].apply(lambda x: re.sub(r'\W+', '', x))
#6 Convert name,genre and type columns to lowercases
anime_df['name'] = anime_df['name'].str.lower()
anime_df['genre'] = anime_df['genre'].str.lower()
anime_df['type'] = anime_df['type'].str.lower()

# Display the cleaned DataFrame
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,kiminonawa,"drama, romance, school, supernatural",movie,1,9.37,200630
1,5114,fullmetalalchemistbrotherhood,"action, adventure, drama, fantasy, magic, mili...",tv,64,9.26,793665
2,28977,gintama,"action, comedy, historical, parody, samurai, s...",tv,51,9.25,114262
3,9253,steinsgate,"sci-fi, thriller",tv,24,9.17,673572
4,9969,gintama,"action, comedy, historical, parody, samurai, s...",tv,51,9.16,151266
